<a href="https://colab.research.google.com/github/ftp1/deeplearning/blob/master/bert_try1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 757kB 2.7MB/s 
     |████████████████████████████████| 890kB 14.7MB/s 
     |████████████████████████████████| 1.1MB 16.5MB/s 
     |████████████████████████████████| 3.0MB 25.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=d92022dcc16a6cc4fac0bf24afd2a141e1b2ff43f96c34df59f8dda9d3d61d19
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
#Importing Libraries

from __future__ import print_function
import numpy as np
import pandas as pd
#import tensorflow as tf
#from tensorflow import keras
#from keras.preprocessing import sequence

#import sys

#sys.setrecursionlimit(100000)

import os

# Intialize variables and load data from labelled dataset
TRAIN_DATA_URL = "https://raw.githubusercontent.com/ftp1/deeplearning/master/sentiment-train.csv"
TEST_DATA_URL = "https://raw.githubusercontent.com/ftp1/deeplearning/master/sentiment-eval.csv"

train_data=pd.read_csv(TRAIN_DATA_URL)
test_data=pd.read_csv(TEST_DATA_URL)
# Convert label to 0/1
num_label = {'Positive': 1,'Negative': 0} 
train_data.label = [num_label[item] for item in train_data.label]

print(train_data)
train_data.text.replace(regex=True, to_replace=r'http\S+',value='')

maxlen_train = train_data.text.str.len().max()
maxlen_test = test_data.text.str.len().max()
max_length = max(maxlen_train, maxlen_test)

X_train = train_data.loc[:, 'text'].values
Y_train = train_data.loc[:, 'label'].values

X_test = test_data.loc[:, 'text'].values
#print(X_test)

      label                                               text
0         0  No one cares about marketing slides - a techni...
1         1  Are all three hosts providing storage/capacity...
2         0  would loved to had managed to get down to the ...
3         0  Vending machine at work is out of Dasani water...
4         1  RT @VMwareEdu: Paul Maritz, CEO and President ...
...     ...                                                ...
1895      1  It's Friday - and that means the latest episod...
1896      1  Your Augmentix A+1950 appears to be nothing mo...
1897      0  Eating breakfast. I've attempted to do my make...
1898      1  vSAN 6.7 update offers enhanced health checks....
1899      0         oh no this is going to be on TMZ tomorrow.

[1900 rows x 2 columns]


In [13]:
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)


X_train_new = []
tokens = []
for text in X_train:
  marked_text = "[CLS] " + text + " [SEP]"
  tokenized_text = tokenizer.tokenize(marked_text)
  tokens.append(tokenized_text)

print(tokens)
print(list(tokenizer.vocab.keys())[5000:5020])

[['[CLS]', 'No', 'one', 'cares', 'about', 'marketing', 'slides', '-', 'a', 'technical', 'H', '##OW', 'TO', 'is', 'a', 'lot', 'better', '[SEP]'], ['[CLS]', 'Are', 'all', 'three', 'hosts', 'providing', 'storage', '/', 'capacity', '?', 'Or', 'is', 'one', 'a', 'witness', '?', 'If', 'all', 'three', 'have', 'capacity', 'drives', ',', 'what', "'", 'd', 'you', 'do', 'with', 'the', 'witness', '?', '(', 'and', 'with', 'them', 'being', 'N', '##UC', '##s', ',', 'just', 'how', 'are', 'you', 'handling', 'capacity', 'and', 'cache', '?', ')', 'I', "'", 'm', 'really', 'wondering', 'about', 'two', '-', 'node', 'performance', ',', 'as', 'I', 'currently', 'have', 'three', 'hosts', 'but', 'one', 'of', 'them', 'has', 'Free', '##NA', '##S', 'virtual', '##ized', 'and', 'all', 'the', 'disk', 'slots', 'on', 'that', 'host', 'are', 'dedicated', 'for', 'that', '.', 'I', 'could', 'add', 'a', 'D', '##AS', 'shelf', 'or', 'something', ',', 'but', 'I', 'am', 'very', 'curious', 'what', 'it', "'", 's', 'like', 'running',